In [44]:
"""
倫敦房價預測 - 混合模型（趨勢分析 + 機器學習）
使用時間序列特徵結合機器學習進行房價預測
"""

import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.pipeline import Pipeline
from statsmodels.tsa.deterministic import DeterministicProcess, CalendarFourier
from sklearn.linear_model import Ridge
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
import optuna
from tqdm import tqdm
from sklearn.model_selection import GridSearchCV, KFold, StratifiedKFold

def objective(trial, X_encoded, y, trend_cols, machine_cols_encoded, all_columns_encoded):
    """
    Optuna objective function for pre-encoded, GPU-ready data.
    This is the high-performance version.
    """
    params = {
        "device": "gpu",
        "n_estimators": trial.suggest_int("n_estimators", 2000, 6000, step=500),
        "max_depth": trial.suggest_int("max_depth", 6, 12),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2, log=True),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-2, 10.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-2, 10.0, log=True),
        "random_state": 42,
        "verbose": -1
    }

    # 直接使用 HybridModel，不再需要 Pipeline
    model = HybridModel(
        trend_model=Ridge(alpha=0.1),
        machine_model=LGBMRegressor(**params),
        trend_cols=trend_cols,
        machine_cols=machine_cols_encoded,
        all_columns=all_columns_encoded
    )

    price_bins = pd.qcut(y, q=10, labels=False, duplicates='drop')
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
    scores = []
    # 序列化執行迴圈，這是 GPU 最高效的工作方式
    for train_idx, val_idx in cv.split(X_encoded, price_bins):
        X_train_fold, X_val_fold = X_encoded.iloc[train_idx], X_encoded.iloc[val_idx]
        y_train_fold, y_val_fold = y.iloc[train_idx], y.iloc[val_idx]
        
        model.fit(X_train_fold, y_train_fold)
        
        preds = model.predict(X_val_fold)
        fold_score = mean_absolute_error(y_val_fold, preds)
        scores.append(fold_score)

    return np.mean(scores)

def create_time_features(data_list):
    """創建時間相關特徵"""
    print("創建時間特徵...")
    for data in data_list:
        # 創建時間索引
        data['time'] = pd.to_datetime(dict(
            year=data['sale_year'], 
            month=data['sale_month'], 
            day=15
        ))
        data['time'] = data['time'].dt.to_period('M')
        
        # 創建數值型時間特徵
        data['time_numeric'] = (
            (data['time'].dt.to_timestamp() - data['time'].min().to_timestamp()) / 
            np.timedelta64(1, 'D')
        )
    
    return data_list


def preprocess_address_features(data_list):
    """處理地址相關特徵"""
    print("處理地址特徵...")
    for data in data_list:
        # 提取街道資訊
        data['street'] = data['fullAddress'].apply(
            lambda address: ' '.join(address.split(',')[-3].split(' ')[-2:])
        )
        
        # 處理郵遞區號
        data['postcode'] = data['postcode'].apply(
            lambda postcode: postcode.split(' ')[1]
        )
        
        # 移除國家欄位（所有資料都是同一個國家）
        data.drop('country', axis=1, inplace=True)
    
    return data_list

def engineer_address_features(data_list):
    """從 fullAddress 中提取更豐富的特徵"""
    print("進行高級地址特徵工程...")
    
    for df in data_list:
        # 將地址轉為小寫以便搜索
        address_lower = df['fullAddress'].str.lower()

        # 1. 提取街道類型
        df['street_type'] = address_lower.str.extract(r'\b(road|street|avenue|lane|square|drive|court|place|gardens|mews)\b', expand=False).fillna('unknown')

        # 2. 是否為公寓/樓層
        df['is_flat_or_apt'] = address_lower.str.contains(r'flat|apartment|unit|floor|level').astype(int)

        # 3. 提取數字資訊 (可能代表門牌號或公寓號)
        # 提取第一個出現的數字序列
        df['address_number'] = address_lower.str.extract(r'(\d+)').astype(float).fillna(0)

        # 4. 地址長度 (一個代理特徵，更長的地址可能意味著更複雜的建築)
        df['address_length'] = df['fullAddress'].str.len()

        # 5. 關鍵詞計數
        keywords = ['mansion', 'penthouse', 'cottage', 'studio', 'garden', 'park', 'view', 'river', 'new build']
        for keyword in keywords:
            df[f'has_{keyword}'] = address_lower.str.contains(keyword).astype(int)
            
    return data_list

def haversine_distance(lat1, lon1, lat2, lon2):
    """
    計算兩點之間的 haversine 距離（單位：公里）
    """
    R = 6371  # 地球半徑 (km)
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
    return R * 2 * np.arcsin(np.sqrt(a))

def engineer_geo_features(train_df, test_df):
    """
    執行地理特徵工程（已移除所有價格相關特徵，只保留純幾何特徵）。
    """
    from sklearn.cluster import KMeans
    print("-> 開始執行地理特徵工程（僅純幾何特徵）...")
    
    train_df_copy = train_df.copy()
    test_df_copy = test_df.copy()
    
    for df in [train_df_copy, test_df_copy]:
        # === 1. 基本地理數學特徵 ===
        df['lat_lon_ratio'] = df['latitude'] / (df['longitude'] + 1e-9)
        df['lat_lon_product'] = df['latitude'] * df['longitude']

        # === 2. 倫敦重要地標距離特徵 ===
        london_landmarks = {
            'city_center': (51.5074, -0.1278), 'canary_wharf': (51.5055, -0.0195),
            'westminster': (51.4994, -0.1244), 'heathrow': (51.4700, -0.4543)
        }
        for name, (lat, lon) in london_landmarks.items():
            df[f'dist_to_{name}'] = haversine_distance(df['latitude'], df['longitude'], lat, lon)

    # === 3. 地理聚類特徵 (Fit on train, transform both) ===
    # 這一步是安全的，因為它只基於座標，不涉及價格
    coords_train = train_df_copy[['latitude', 'longitude']].values
    coords_test = test_df_copy[['latitude', 'longitude']].values
    cluster_configs = {'geo_cluster_medium': 20, 'geo_cluster_fine': 50}
    
    for name, k in cluster_configs.items():
        kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
        train_df_copy[name] = kmeans.fit_predict(coords_train)
        test_df_copy[name] = kmeans.predict(coords_test)
        
    print("-> 純地理特徵工程完成。")
    return train_df_copy, test_df_copy

def impute_missing_values_with_strategy(data_list, column_name, strategy='most_frequent'):
    """使用指定策略填補缺失值"""
    print(f"填補 {column_name} 的缺失值（策略：{strategy}）...")
    
    # 從訓練資料學習填補策略
    train_data = data_list[0]  # 第一個是訓練資料
    imputer = SimpleImputer(strategy=strategy)
    imputer.fit(train_data[[column_name]])
    
    # 對所有資料集應用填補
    for data in data_list:
        data[column_name] = imputer.transform(data[[column_name]]).ravel()
    
    return data_list


def impute_with_regression(data_list, target_column, feature_column):
    """使用回歸模型填補缺失值"""
    print(f"使用 {feature_column} 預測填補 {target_column} 的缺失值...")
    
    train_data = data_list[0]
    test_data = data_list[1]
    
    # 準備完整的訓練資料
    complete_train_data = train_data.dropna(subset=[target_column, feature_column])
    X_train = complete_train_data[[feature_column]]
    y_train = complete_train_data[target_column]
    
    # 訓練回歸模型
    regression_model = Ridge()
    regression_model.fit(X_train, y_train)
    
    # 填補訓練集的缺失值
    missing_train_mask = train_data[target_column].isna()
    if missing_train_mask.any():
        missing_train_features = train_data.loc[missing_train_mask, [feature_column]]
        train_data.loc[missing_train_mask, target_column] = regression_model.predict(missing_train_features)
    
    # 填補測試集的缺失值
    missing_test_mask = test_data[target_column].isna()
    if missing_test_mask.any():
        missing_test_features = test_data.loc[missing_test_mask, [feature_column]]
        test_data.loc[missing_test_mask, target_column] = regression_model.predict(missing_test_features)
    
    return data_list

def handle_missing_values(data_list):
    """處理所有缺失值"""
    print("開始處理缺失值...")
    
    # 使用最頻繁值填補面積
    data_list = impute_missing_values_with_strategy(data_list, 'floorAreaSqM')
    
    # 使用面積預測浴室數量
    data_list = impute_with_regression(data_list, 'bathrooms', 'floorAreaSqM')
    
    # 使用面積預測臥室數量
    data_list = impute_with_regression(data_list, 'bedrooms', 'floorAreaSqM')
    
    # 使用最頻繁值填補其他類別特徵
    categorical_columns = ['livingRooms', 'tenure', 'propertyType', 'currentEnergyRating']
    for column in categorical_columns:
        data_list = impute_missing_values_with_strategy(data_list, column)
    
    return data_list


def create_time_series_features(train_data, test_data):
    """創建時間序列特徵"""
    print("創建時間序列特徵...")
    
    # 創建確定性過程（趨勢、季節性、週期性）
    deterministic_process = DeterministicProcess(
        index=train_data.index.unique(),
        constant=True,        # 常數項
        seasonal=True,        # 季節性
        order=12,            # 趨勢階數
        drop=True,           # 移除共線性
        additional_terms=[CalendarFourier(freq="QE", order=4)],  # 季度傅立葉項
    )
    
    # 為訓練資料添加時間序列特徵
    time_features_train = deterministic_process.in_sample()
    train_data = train_data.join(time_features_train, how='left')
    
    # 計算預測相關參數
    forecast_origin = train_data.index.max()
    forecast_lead = test_data.index.min() - forecast_origin
    forecast_horizon = test_data.index.max() - test_data.index.min()
    
    print(f"預測起點: {forecast_origin}")
    print(f"領先時間: {forecast_lead.n} 個月")
    print(f"預測範圍: {forecast_horizon.n} 個月")
    
    # 為測試資料添加時間序列特徵
    time_features_test = deterministic_process.out_of_sample(
        steps=forecast_horizon.n + forecast_lead.n
    )
    test_data = test_data.join(time_features_test, how='left')
    test_data.index.name = 'time'
    
    return train_data, test_data, time_features_train.columns.tolist()


def create_additional_features(data_list):
    """創建額外的特徵"""
    print("創建額外特徵...")
    
    for data in data_list:
        # 總房間數 = 臥室 + 起居室
        data['rooms'] = data['bedrooms'] + data['livingRooms']
        # 總房間數 = 臥室 + 起居室
        data['rooms'] = data['bedrooms'] + data['livingRooms']
        
        # 衍生密度特徵
        data['rooms_per_bedroom'] = data['rooms'] / np.maximum(data['bedrooms'], 1)
        data['bath_per_room'] = data['bathrooms'] / np.maximum(data['rooms'], 1)

        # === 新增：創建交叉特徵 ===
        # 將 outcode 和 propertyType 結合，形成更具體的特徵
        # 例如 "SW1_Flat" (SW1區的公寓)
        data['outcode_proptype'] = data['outcode'].astype(str) + "_" + data['propertyType'].astype(str)

        # 將 outcode 和 tenure 結合
        data['outcode_tenure'] = data['outcode'].astype(str) + "_" + data['tenure'].astype(str)
            
    
    return data_list

class CustomEncoder(BaseEstimator, TransformerMixin):
    """
    智能編碼器，自動處理所有類別特徵，並對特定特徵進行專門處理。
    """
    def __init__(self):
        self.target_mean_encoders = {}
        self.fallback_values = {}
        self.bin_encoders = {}
        self.ordinal_encoders = {}
        self.object_cols_to_encode = []

    def fit(self, X, y=None):
        X_copy = X.copy()
        X_copy['price'] = y
        
        # 1. 智能識別所有需要進行目標編碼的 object 欄位
        #    排除 'currentEnergyRating'，因其需要特殊的順序編碼
        self.object_cols_to_encode = [
            col for col in X_copy.select_dtypes(include='object').columns
            if col != 'currentEnergyRating'
        ]
        
        for feature in self.object_cols_to_encode:
            self.target_mean_encoders[feature] = X_copy.groupby(feature)['price'].mean()
            self.fallback_values[feature] = self.target_mean_encoders[feature].mean()
        
        # 2. 對緯度和經度進行分箱編碼 (保持不變)
        latitude_bins = pd.cut(X_copy['latitude'], bins=10, retbins=True)[1]
        self.bin_encoders['latitudeBins'] = latitude_bins
        longitude_bins = pd.cut(X_copy['longitude'], bins=10, retbins=True)[1]
        self.bin_encoders['longitudeBins'] = longitude_bins
        
        # 3. 對能源評級進行特殊的順序編碼 (保持不變)
        energy_rating_order = [['G', 'F', 'E', 'D', 'C', 'B', 'A']]
        present_ratings = X_copy['currentEnergyRating'].unique()
        for r in present_ratings:
            if r not in energy_rating_order[0]:
                energy_rating_order[0].append(r)
        self.ordinal_encoders['currentEnergyRating'] = OrdinalEncoder(
            categories=energy_rating_order,
            handle_unknown='use_encoded_value',
            unknown_value=-1
        ).fit(X_copy[['currentEnergyRating']])
        
        return self

    def transform(self, X):
        X_transformed = X.copy()
        
        # 1. 應用目標編碼
        for feature in self.object_cols_to_encode:
            if feature in X_transformed.columns:
                X_transformed[feature] = X_transformed[feature].map(self.target_mean_encoders[feature])
                X_transformed[feature] = X_transformed[feature].fillna(self.fallback_values[feature])
        
        # 2. 應用分箱編碼
        X_transformed['latitudeBins'] = pd.cut(X_transformed['latitude'], bins=self.bin_encoders['latitudeBins'], include_lowest=True, right=True, labels=False)
        X_transformed['longitudeBins'] = pd.cut(X_transformed['longitude'], bins=self.bin_encoders['longitudeBins'], include_lowest=True, right=True, labels=False)
        
        # 3. 應用順序編碼
        X_transformed['currentEnergyRating'] = self.ordinal_encoders['currentEnergyRating'].transform(
            X_transformed[['currentEnergyRating']]
        )
        
        return X_transformed
    
class HybridModel(BaseEstimator, RegressorMixin):
    """
    混合模型：結合趨勢模型(CPU)和機器學習模型(GPU)
    - 趨勢模型：在 CPU 上處理時間序列特徵
    - 機器學習模型：在 GPU 上處理殘差和其他特徵
    """
    
    def __init__(self, trend_model, machine_model, trend_cols, machine_cols, all_columns):
        self.trend_model = trend_model
        self.machine_model = machine_model
        self.trend_cols = trend_cols
        self.machine_cols = machine_cols
        self.all_columns = all_columns
        self.machine_cols_encoded_ = None

    def fit(self, X, y):
        """訓練混合模型"""
        # 管道已對 X 進行編碼，X 是 DataFrame
        y_log = np.log1p(y)
        
        trend_features = X[self.trend_cols]
        
        # 機器學習特徵是除了趨勢特徵之外的所有特徵
        machine_cols_encoded = [col for col in X.columns if col not in self.trend_cols]
        self.machine_cols_encoded_ = machine_cols_encoded # 儲存以供 predict 使用
        machine_features = X[machine_cols_encoded]
        
        # 1. 訓練趨勢模型 (在 CPU 上)
        self.trend_model.fit(trend_features, y_log)
        
        # 2. 計算殘差 (在 CPU 上)
        trend_predictions = self.trend_model.predict(trend_features)
        residual = y_log - trend_predictions
        
        # 3. 用機器學習模型學習殘差 (LGBM 在 GPU 上)
        self.machine_model.fit(machine_features, residual)
        
        return self

    # ...緊接著 fit 方法的結尾...

    def predict_components(self, X):
        """預測趨勢和殘差的組成部分（在對數空間中）"""
        # 管道已對 X 進行編碼，X 是 DataFrame
        trend_features = X[self.trend_cols]
        machine_features = X[self.machine_cols_encoded_]
        
        # 趨勢預測(CPU)和機器學習預測(GPU)
        trend_predictions = self.trend_model.predict(trend_features)
        machine_predictions = self.machine_model.predict(machine_features)
        
        return trend_predictions, machine_predictions


    def predict(self, X):
        """進行預測"""
        trend_predictions, machine_predictions = self.predict_components(X)
        
        # 組合預測結果
        combined_predictions = trend_predictions + machine_predictions
        return np.expm1(combined_predictions)

def better_features(features, target_series, is_train, config, columns_to_combine=None, quick_test=False):
    """
    自動化生成和篩選算術組合特徵。
    - 訓練模式 (is_train=True): 探索新特徵，評估其與目標的相關性及與現有特徵的冗餘性，
      並將有效的特徵組合儲存到 config 中。
    - 推論模式 (is_train=False): 從 config 中讀取已儲存的特徵組合，並應用到數據集上。
    """
    print("-> 執行特徵組合...")

    if quick_test:
        print("    啟用快速測試模式，僅使用少量核心特徵進行組合。")
        core_features_for_combination = ['floorAreaSqM', 'total_rooms', 'lat_lon_ratio']
        features_to_use = [f for f in core_features_for_combination if f in features.columns]
    else:
        # 確保只選擇數值型特徵
        features_to_use = features.select_dtypes(include=np.number).columns.tolist()
        # 移除 'index' 列（如果存在）
        if 'index' in features_to_use:
            features_to_use.remove('index')

    if len(features_to_use) < 2:
        print("    可用於組合的數值特徵少於2個，跳過此步驟。")
        if is_train:
            config['better_features_list'] = [] # 確保鍵存在
        return features, config

    if is_train:
        if target_series is None:
            print("    在訓練模式下未提供目標變數，跳過特徵組合。")
            config['better_features_list'] = [] # 確保鍵存在
            return features, config

        # --- 使用 NumPy 進行向量化計算以大幅提高性能 ---
        print("    初始化 NumPy 矩陣以加速相關性計算...")
        
        target_vector = target_series.values
        target_vector_norm = (target_vector - np.mean(target_vector)) / np.std(target_vector)

        existing_features_matrix = features[features_to_use].values
        
        mean_existing = np.mean(existing_features_matrix, axis=0)
        std_existing = np.std(existing_features_matrix, axis=0)
        std_existing[std_existing == 0] = 1
        normalized_existing_matrix = (existing_features_matrix - mean_existing) / std_existing
        
        n_rows = len(target_vector)
        new_combinations_list = [] 
        new_features_to_add = []

        print("    正在搜索最佳算術組合 (使用向量化)...")
        
        for i in tqdm(range(len(features_to_use)), desc="    組合特徵搜索", leave=False):
            for j in range(i, len(features_to_use)):
                col1 = features_to_use[i]
                col2 = features_to_use[j]

                for op in ['*', '/', '-', '+']:
                    if col1 == col2 and op in ['-']: continue
                    if i > j and op in ['+', '*']: continue

                    new_feature_name = f'{col1}_{op}_{col2}'
                    
                    if op == '+': new_feature_series = features[col1] + features[col2]
                    elif op == '-': new_feature_series = features[col1] - features[col2]
                    elif op == '*': new_feature_series = features[col1] * features[col2]
                    elif op == '/': new_feature_series = features[col1] / (features[col2] + 1e-9)
                    
                    if new_feature_series.isnull().any() or np.isinf(new_feature_series).any():
                        continue

                    new_feature_vector = new_feature_series.values
                    
                    mean_new = np.mean(new_feature_vector)
                    std_new = np.std(new_feature_vector)
                    if std_new < 1e-9: continue
                    
                    new_feature_norm = (new_feature_vector - mean_new) / std_new
                    correlation_with_target = np.dot(target_vector_norm, new_feature_norm) / n_rows
                    
                    if abs(correlation_with_target) < 0.05:
                        continue

                    correlations_with_existing = np.dot(new_feature_norm.T, normalized_existing_matrix) / n_rows
                    if np.max(np.abs(correlations_with_existing)) > 0.95:
                        continue
                    
                    # 將 new_feature_name 賦給 Series，以便後續 concat
                    new_feature_series.name = new_feature_name 
                    new_features_to_add.append(new_feature_series) # <--- 不直接賦值，而是加入 list
                    
                    
                    # features[new_feature_name] = new_feature_series
                    normalized_existing_matrix = np.c_[normalized_existing_matrix, new_feature_norm]
                    new_combinations_list.append((col1, col2, op))
        
        # 在所有迴圈結束後，一次性合併所有新特徵
        if new_features_to_add:
            features = pd.concat([features] + new_features_to_add, axis=1)
        
        print(f"    發現 {len(new_combinations_list)} 個新組合特徵。")
        config['better_features_list'] = new_combinations_list
        return features, config

    else: # is_train == False
        if 'better_features_list' not in config or not config['better_features_list']:
            print("    在非訓練模式下，未找到已儲存的特徵組合，跳過此步驟。")
            return features, config
        
        combinations = config['better_features_list']
        new_cols_dict = {}
        print(f"    應用 {len(combinations)} 個已儲存的特徵組合。")
        for col1, col2, op in combinations:
            new_col_name = f'{col1}_{op}_{col2}'
            if op == '+': new_cols_dict[new_col_name] = features[col1] + features[col2]
            elif op == '-': new_cols_dict[new_col_name] = features[col1] - features[col2]
            elif op == '*': new_cols_dict[new_col_name] = features[col1] * features[col2]
            elif op == '/': new_cols_dict[new_col_name] = features[col1] / (features[col2] + 1e-9)
        
        # 一次性從字典創建 DataFrame 並合併
        if new_cols_dict:
            new_features_df = pd.DataFrame(new_cols_dict, index=features.index)
            features = pd.concat([features, new_features_df], axis=1)

        return features.copy(), config

# 一個自定義的元模型，它的唯一工作就是將輸入的預測值相加
class SummingRegressor(BaseEstimator, RegressorMixin):
    def fit(self, X, y=None):
        # 不需要學習任何東西
        return self

    def predict(self, X):
        # X 是一個包含兩欄的 numpy array (趨勢預測, 殘差預測)
        # 我們沿著欄位（axis=1）將它們相加
        return np.sum(X, axis=1)


class StackingModel(BaseEstimator, RegressorMixin):
    """
    Stacking 集成模型:
    """
    def __init__(self, base_models, meta_model, n_splits=5):
        self.base_models = base_models  # 注意這裡從 base_model 改為 base_models
        self.meta_model = meta_model
        self.n_splits = n_splits
        self.trained_models_ = [] # 存儲每個 fold 訓練好的所有基礎模型

    def fit(self, X, y):
        print(f"\n=== 開始 Stacking 訓練 ({len(self.base_models)} 種基礎模型) ===")
        
        cv = KFold(n_splits=self.n_splits, shuffle=True, random_state=42)
        
        # 建立一個列表，用來存放每個 fold 的模型列表
        self.trained_models_ = [[] for _ in range(self.n_splits)]
        
        # 我們仍然需要 OOF 預測來評估，但現在維度變了
        # 每一種基礎模型都會產生一組 OOF 預測
        oof_predictions_list = [np.zeros_like(y, dtype=float) for _ in self.base_models]

        for i, (train_idx, val_idx) in enumerate(tqdm(cv.split(X, y), total=self.n_splits, desc="    訓練 Fold 模型")):
            X_train_fold, X_val_fold = X.iloc[train_idx], X.iloc[val_idx]
            y_train_fold = y.iloc[train_idx]
            
            # 對於每一種基礎模型，都進行訓練和預測
            for model_idx, base_model in enumerate(self.base_models):
                fold_model = clone(base_model)
                fold_model.fit(X_train_fold, y_train_fold)
                self.trained_models_[i].append(fold_model) # 存儲訓練好的模型
                
                # 在驗證集上產生預測 (注意：fold_model.predict 直接返回價格)
                val_preds = fold_model.predict(X_val_fold)
                oof_predictions_list[model_idx][val_idx] = val_preds
        
        print("    OOF 特徵生成完畢。")
        
        # 評估簡單平均模型的性能
        print("\n    --- 評估 OOF 上的簡單平均模型 ---")
        # 將所有模型的 OOF 預測取平均
        final_oof_preds = np.mean(oof_predictions_list, axis=0)
        simple_avg_mae = mean_absolute_error(y, final_oof_preds)
        print(f"    -> OOF 簡單平均 MAE: {simple_avg_mae:.4f}")
        
        print("=== Stacking 訓練完成。 ===")
        return self

    def predict(self, X):
        print("    Ensemble 預測 (Averaging)...")
        
        predictions_list = []
        # 遍歷每一個 Fold
        for fold_models in self.trained_models_:
            # 在該 Fold 中，遍歷所有類型的基礎模型
            for model in fold_models:
                predictions_list.append(model.predict(X))
        
        # 對所有預測（n_splits * n_base_models 個）取平均
        final_predictions = np.mean(predictions_list, axis=0)
        
        return final_predictions
    

def prepare_features(train_data, test_data, time_series_features):
    """
    準備特徵集合的最終、最穩健版本。
    """
    print("準備特徵集合...")

    # --- 修正 NameError ---
    # 在使用前，先定義 trend_features
    trend_features = time_series_features
    
    # 在最開始就丟棄高噪聲的原始文字欄位
    raw_text_cols_to_drop = ['fullAddress']
    train_data = train_data.drop(columns=[col for col in raw_text_cols_to_drop if col in train_data.columns])
    test_data = test_data.drop(columns=[col for col in raw_text_cols_to_drop if col in test_data.columns])
    
    # 準備訓練特徵和目標
    X_train = train_data.drop('price', axis=1)
    y_train = train_data['price']

    # 機器學習特徵 *名稱列表*
    all_cols = X_train.columns
    machine_learning_features = [
        col for col in all_cols if col not in trend_features
    ]
    print(f"定義了 {len(machine_learning_features)} 個機器學習特徵。")

    # 標準化時間序列特徵
    scaler = StandardScaler()
    features_to_scale = [f for f in trend_features if f in X_train.columns and f in test_data.columns]
    
    if features_to_scale:
        X_train.loc[:, features_to_scale] = scaler.fit_transform(X_train[features_to_scale])
        test_data.loc[:, features_to_scale] = scaler.transform(test_data[features_to_scale])
    
    # 返回五個值，包括更新後的 test_data
    return X_train, y_train, trend_features, machine_learning_features, test_data.copy()


def create_and_tune_model(X_train, y_train, trend_features, machine_learning_features):
    """創建並調優模型"""
    print("創建混合模型並進行超參數調優...")
    
    # 定義模型管道
    model_pipeline = {
        'HybridModel': Pipeline([
            ('Encoder', CustomEncoder()),
            ('Model', HybridModel(
                trend_model=Ridge(),
                machine_model=LGBMRegressor(device='gpu', verbose=-1),
                trend_cols=trend_features,
                machine_cols=machine_learning_features,
                all_columns=X_train.columns
            ))
        ]),
    }
    
    # 定義超參數搜索空間
    hyperparameter_grid = {
        'HybridModel': {
            'Model__trend_model__alpha': [0.01, 0.1],
            'Model__machine_model__n_estimators': [300,600],
            'Model__machine_model__max_depth': [4,6,8],
            'Model__machine_model__learning_rate': [0.01, 0.005, 0.1],
        }
    }
    
    # 進行網格搜索
    best_models = {}
    for model_name, pipeline in model_pipeline.items():
        print(f"調優 {model_name}...")
        cv = KFold(n_splits=5, shuffle=True, random_state=42)
        grid_search = GridSearchCV(
            pipeline, 
            hyperparameter_grid[model_name], 
            cv=cv, 
            scoring='neg_mean_absolute_error', 
            n_jobs=-1, 
            verbose=2, 
            error_score='raise'
        )
        
        grid_search.fit(X_train, y_train)
        
        print(f"{model_name} 最佳參數: {grid_search.best_params_}")
        print(f"{model_name} 最佳 MAE: {-grid_search.best_score_:.4f}")
        
        best_models[model_name] = grid_search.best_estimator_
    
    return best_models

def create_ensemble_model(best_models, X_train, y_train):
    """創建集成模型"""
    print("創建集成模型...")
    
    # 準備集成模型的估計器列表
    ensemble_estimators = [
        ('HybridModel', best_models['HybridModel']),
    ]
    
    # 創建投票回歸器
    ensemble_model = VotingRegressor(estimators=ensemble_estimators)
    ensemble_model.fit(X_train, y_train)
    
    print(f"集成模型: {ensemble_model}")
    
    return ensemble_model


def evaluate_model(model, X_train, y_train):
    """評估模型性能"""
    print("評估模型性能...")
    
    # 預測訓練集
    train_predictions = model.predict(X_train)
    
    # 計算評估指標
    mae = mean_absolute_error(y_train, train_predictions)
    rmse = mean_squared_error(y_train, train_predictions, squared=False)
    r2 = r2_score(y_train, train_predictions)
    
    print(f"[訓練集] MAE: {mae:.4f}, RMSE: {rmse:.4f}, R²: {r2:.4f}")
    
    return mae, rmse, r2


def run_optuna_tuning(X_train_encoded, y_train, trend_features, machine_learning_features_encoded, all_columns_encoded, n_trials=100):
    """使用 Optuna 進行超參數調優（使用預處理數據）"""
    print("開始 Optuna 超參數調優 (使用預處理數據)...")
    
    study = optuna.create_study(direction='minimize')
    
    study.optimize(
        lambda trial: objective(trial, X_train_encoded, y_train, trend_features, machine_learning_features_encoded, all_columns_encoded),
        n_trials=n_trials,
        show_progress_bar=True
    )
    
    print(f"最佳參數: {study.best_params}")
    print(f"最佳 MAE (CV): {study.best_value:.4f}")
    
    # 使用最佳參數創建並訓練最終模型
    best_params = study.best_params
    # 確保參數字典不包含 Optuna 內部使用的 'n_estimators' 等鍵，如果它們不適用於最終模型
    final_model = HybridModel(
        trend_model=Ridge(alpha=0.1),
        machine_model=LGBMRegressor(device='gpu', **best_params),
        trend_cols=trend_features,
        machine_cols=machine_learning_features_encoded,
        all_columns=all_columns_encoded
    )
    
    final_model.fit(X_train_encoded, y_train)
    
    return final_model

def run_optuna_tuning(X_train_encoded, y_train, trend_features, machine_learning_features_encoded, all_columns_encoded, n_trials=100):
    """使用 Optuna 進行超參數調優（使用預處理數據）"""
    print("開始 Optuna 超參數調優 (使用預處理數據)...")
    
    study = optuna.create_study(direction='minimize')
    
    study.optimize(
        lambda trial: objective(trial, X_train_encoded, y_train, trend_features, machine_learning_features_encoded, all_columns_encoded),
        n_trials=n_trials,
        show_progress_bar=True
    )
    
    print(f"最佳參數: {study.best_params}")
    print(f"最佳 MAE (CV): {study.best_value:.4f}")
    
    # 使用最佳參數創建並訓練最終模型
    best_params = study.best_params
    # 確保參數字典不包含 Optuna 內部使用的 'n_estimators' 等鍵，如果它們不適用於最終模型
    final_model = HybridModel(
        trend_model=Ridge(alpha=0.1),
        machine_model=LGBMRegressor(device='gpu', **best_params),
        trend_cols=trend_features,
        machine_cols=machine_learning_features_encoded,
        all_columns=all_columns_encoded
    )
    
    final_model.fit(X_train_encoded, y_train)
    
    return final_model

def train_with_fixed_params(X_train, y_train, trend_features, machine_learning_features, best_params):
    """使用一組固定的超參數直接訓練最終模型"""
    print("使用固定參數進行最終模型訓練...")
    print(f"使用參數: {best_params}")

    # 創建包含 XGBoost 的模型管道
    model_pipeline = Pipeline([
        ('Encoder', CustomEncoder()),
        ('Model', HybridModel(
            trend_model=Ridge(), # alpha 將在下一步被設定
            machine_model=LGBMRegressor(device='gpu', random_state=42), # 其他參數將在下一步被設定
            trend_cols=trend_features,
            machine_cols=machine_learning_features,
            all_columns=X_train.columns
        ))
    ])

    # 使用 set_params() 將您提供的參數應用到管道中
    model_pipeline.set_params(**best_params)

    # 訓練模型
    model_pipeline.fit(X_train, y_train)

    print("模型訓練完成。")
    return model_pipeline

In [45]:
def load_and_prepare_data():
    """載入並準備訓練和測試資料"""
    print("載入資料...")
    train_df = pd.read_csv('/kaggle/input/new-london-house-price/train.csv')
    test_df = pd.read_csv('/kaggle/input/new-london-house-price/test.csv')
    
    # 為測試集添加空的價格欄位
    test_df['price'] = np.nan
    
    return train_df, test_df

def generate_submission(model, test_data):
    """生成提交檔案"""
    print("生成提交檔案...")
    
    # 載入提交模板
    submission = pd.read_csv('/kaggle/input/new-london-house-price/sample_submission.csv')
    
    # 進行預測
    test_features = test_data.drop('price', axis=1)
    submission['price'] = model.predict(test_features)
    
    # 儲存提交檔案
    submission.to_csv('submission_RandomKFold.csv', index=False)
    print("提交檔案已儲存為 submission_RandomKFold.csv")

In [ ]:
def main():
    """主要執行流程（已整合所有特徵工程和 Stacking 架構）"""
    print("=== 倫敦房價預測 - 最終整合版 ===")
    
    # 1. 載入資料
    train_df, test_df = load_and_prepare_data()

    # --- 核心特徵工程管線 (步驟 2 到 7.5) ---
    data_list = create_time_features([train_df, test_df])
    train_df, test_df = data_list[0], data_list[1]
    train_df = train_df.set_index('time')
    test_df = test_df.set_index('time')
    data_list = preprocess_address_features([train_df, test_df])
    data_list = engineer_address_features(data_list)
    train_df, test_df = data_list[0], data_list[1]
    data_list = handle_missing_values([train_df, test_df])
    train_df, test_df = data_list[0], data_list[1]
    train_df, test_df, time_series_features = create_time_series_features(train_df, test_df)
    data_list = create_additional_features([train_df, test_df])
    train_df, test_df = data_list[0], data_list[1]
    train_df, test_df = engineer_geo_features(train_df, test_df)
    
    print("\n--- 開始自動化特徵組合 ---")
    feature_combo_config = {}
    y_train_backup = train_df['price'].copy()
    train_df, feature_combo_config = better_features(
        features=train_df.drop(columns=['price']), 
        target_series=y_train_backup,
        is_train=True,
        config=feature_combo_config
    )
    train_df = train_df.copy()
    train_df['price'] = y_train_backup
    test_df, _ = better_features(
        features=test_df, 
        target_series=None,
        is_train=False,
        config=feature_combo_config
    )
    print("--- 自動化特徵組合完成 ---\n")
    
    # 8. 準備最終特徵
    X_train, y_train, trend_features, machine_learning_features, test_df = prepare_features(
        train_df, test_df, time_series_features
    )
    
    # 9. 【關鍵】: 在所有流程外部預先編碼
    encoder = CustomEncoder()
    X_train_encoded = encoder.fit_transform(X_train, y_train)
    # test_df_encoded = encoder.transform(test_df)

    # 10. 定義模型參數 (使用你找到的最佳參數)
    lgbm_params = {
        'n_estimators': 2500, 'max_depth': 7, 'learning_rate': 0.08840671722264341, 
        'subsample': 0.9097714385340092, 'colsample_bytree': 0.8595636633905605, 
        'reg_alpha': 4.2313700227253115, 'reg_lambda': 0.008959635134224476
    }
    
    # 為 XGBoost 設定一組合理的參數 (可以用之前實驗的)
    xgb_params = {
        'n_estimators': 2500, 'max_depth': 7, 'learning_rate': 0.08840671722264341, 
        'subsample': 0.9097714385340092, 'colsample_bytree': 0.8595636633905605, 
        'reg_alpha': 4.2313700227253115, 'reg_lambda': 0.008959635134224476
    }

    # 11. 【新架C構】創建並訓練 Stacking 模型
    
    # 基礎模型 1: LightGBM
    base_model_lgbm = Pipeline([
            ('Encoder', CustomEncoder()),
            ('Model', HybridModel(
                trend_model=Ridge(alpha=0.1),
                machine_model=LGBMRegressor(device='gpu', verbose=-1, **lgbm_params),
                trend_cols=trend_features,
                machine_cols=machine_learning_features,
                all_columns=X_train.columns
            ))
        ])
        
    # 基礎模型 2: XGBoost
    base_model_xgb = Pipeline([
            ('Encoder', CustomEncoder()),
            ('Model', HybridModel(
                trend_model=Ridge(alpha=0.1),
                machine_model=XGBRegressor(**xgb_params),
                trend_cols=trend_features,
                machine_cols=machine_learning_features,
                all_columns=X_train.columns
            ))
        ])
        
    meta_model_instance = SummingRegressor()
    final_model1 = StackingModel(
        base_models=[
            base_model_lgbm, 
            base_model_xgb],
        meta_model=meta_model_instance,
        n_splits=5
    )
    final_model1.fit(X_train, y_train)
    
    
    print("\n--- stacking模型評估 ---")
    evaluate_model(final_model1, X_train, y_train)


    
    print("\n--- 生成提交文件 ---")
    generate_submission(final_model1, test_df)
    
    print("\n=== 程序執行完成 ===")


# 執行主程序
if __name__ == "__main__":
    main()

=== 倫敦房價預測 - 最終整合版 ===
載入資料...
創建時間特徵...
處理地址特徵...
進行高級地址特徵工程...
開始處理缺失值...
填補 floorAreaSqM 的缺失值（策略：most_frequent）...
使用 floorAreaSqM 預測填補 bathrooms 的缺失值...
使用 floorAreaSqM 預測填補 bedrooms 的缺失值...
填補 livingRooms 的缺失值（策略：most_frequent）...
填補 tenure 的缺失值（策略：most_frequent）...
填補 propertyType 的缺失值（策略：most_frequent）...
填補 currentEnergyRating 的缺失值（策略：most_frequent）...
創建時間序列特徵...
預測起點: 2023-12
領先時間: 1 個月
預測範圍: 7 個月
創建額外特徵...
-> 開始執行地理特徵工程（僅純幾何特徵）...
-> 純地理特徵工程完成。

--- 開始自動化特徵組合 ---
-> 執行特徵組合...
    初始化 NumPy 矩陣以加速相關性計算...
    正在搜索最佳算術組合 (使用向量化)...


    發現 307 個新組合特徵。
-> 執行特徵組合...
    應用 307 個已儲存的特徵組合。
--- 自動化特徵組合完成 ---

準備特徵集合...
定義了 349 個機器學習特徵。

=== 開始 Stacking 訓練 (1 種基礎模型) ===


    訓練 Fold 模型:   0%|          | 0/5 [00:00<?, ?it/s]